# GPU

查看显卡信息

In [1]:
!nvidia-smi

Thu Jul 27 10:41:15 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.18                 Driver Version: 531.18       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650       WDDM | 00000000:01:00.0  On |                  N/A |
| N/A   53C    P0               14W /  N/A|    759MiB /  4096MiB |      8%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

计算设备

In [2]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

查询可用gpu的数量

In [3]:
torch.cuda.device_count()

0

这两个函数允许我们在不存在所需所有GPU的情况下运行代码

In [4]:
def try_gpu(i=0):  
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cpu'), device(type='cpu'), [device(type='cpu')])

查询张量所在的设备

In [5]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

存储在GPU上

In [6]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]])

第二个GPU上创建一个随机张量

In [7]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.1195, 0.6603, 0.0034],
        [0.1097, 0.6371, 0.0915]])

要计算`X + Y`，我们需要决定在哪里执行这个操作

In [10]:
Z = X.cuda(1)
print(X)
print(Z)

AssertionError: Torch not compiled with CUDA enabled

现在数据在同一个GPU上（`Z`和`Y`都在），我们可以将它们相加

In [9]:
Y + Z

NameError: name 'Z' is not defined

In [ ]:
Z.cuda(1) is Z

True

神经网络与GPU

In [ ]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

net(X)

tensor([[-0.0605],
        [-0.0605]], device='cuda:0', grad_fn=<AddmmBackward0>)

确认模型参数存储在同一个GPU上

In [ ]:
net[0].weight.data.device

device(type='cuda', index=0)